In [1]:
from bootstrap import bs_means_diff, bs_stdofmean
from decadal_stats import collect_data, compare_periods
from itertools import compress
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
#Path to RDA results
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"

period = [201,273]
years = [1951,1979]

In [3]:
compare_periods([[1951,1979],[1980,2007]],[201,273], primary_only = True, \
                latrange=[27,99])

/Users/Siwen/RDA/decadal_stats.py:50: RuntimeWarning: invalid value encountered in greater
  lat_1_pass = (lat_1_out > latrange[0]) & (lat_1_out < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:50: RuntimeWarning: invalid value encountered in less
  lat_1_pass = (lat_1_out > latrange[0]) & (lat_1_out < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:51: RuntimeWarning: invalid value encountered in greater
  lat_2_pass = (lat_2_out > latrange[0]) & (lat_2_out < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:51: RuntimeWarning: invalid value encountered in less
  lat_2_pass = (lat_2_out > latrange[0]) & (lat_2_out < latrange[1])


{'frequency': {},
 'intensity': {'diff_p1p2': 0.61353578255149444,
  'mean_p1': 23.491198109399697,
  'mean_p2': 24.104733891951192,
  'pval': 0.88471152884711524,
  'std_p1': 0.34500613764100591,
  'std_p2': 0.38182347545921302},
 'latitude': {'diff_p1p2': -0.69520139788962609,
  'mean_p1': 33.61262422043778,
  'mean_p2': 32.917422822548154,
  'pval': 0.00019998000199980003,
  'std_p1': 0.13005036050812349,
  'std_p2': 0.13029135527087402}}